In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import gym

In [23]:
class ReplayBuffer():
    def __init__(self,memsize,state_dim,action_dim):
        self.memsize=memsize
        self.cntr=0
        self.s=np.zeros((self.memsize,state_dim))
        self.a=np.zeros((self.memsize,action_dim))
        self.r=np.zeros(self.memsize)
        self.s_=np.zeros((self.memsize,state_dim))
        self.d=np.zeros(self.memsize,dtype=np.bool8)
        
    def add(self,state,action,reward,next_state,done):
        index=self.cntr%self.memsize
        self.s[index]=state
        self.a[index]=action
        self.r[index]=reward
        self.s_[index]=next_state
        self.d[index]=done
        self.cntr+=1
        
    def sample(self,batch_size):
        maxmem=min(self.memsize,self.cntr)
        index=np.random.choice(maxmem,batch_size,replace=True)
        states=self.s[index]
        actions=self.a[index]
        rewards=self.r[index]
        next_states=self.s_[index]
        dones=self.d[index]
        
        return states,actions,rewards,next_states,dones

In [79]:
class Actor(nn.Module):
    def __init__(self, state_dim,hidden_dim,max_action,max_log_std=2):
        super(Actor,self).__init__()
        self.fc1=nn.Linear(state_dim,hidden_dim)
        self.fc2=nn.Linear(hidden_dim,hidden_dim)
        self.mu_head=nn.Linear(hidden_dim,1)
        self.log_std_head=nn.Linear(hidden_dim,1)
        self.max_action=max_action
        self.reparam_noise=1e-6
        
        self.max_log_std=max_log_std
        
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        mu=self.mu_head(x)
        log_std_head=self.log_std_head(x)
        log_std_head=torch.clamp(log_std_head,self.reparam_noise,self.max_log_std)
        return mu,log_std_head
    
    def select_action(self,state):
        mu,log_std_head=self.forward(state)
        probs=torch.distributions.Normal(mu,log_std_head)
        action_=probs.sample()
        action=torch.tanh(action_)*torch.tensor(self.max_action)
        log_probs=probs.log_prob(action_)
        log_probs-=torch.log(1-action.pow(2)+self.reparam_noise)
        #print(log_probs)
        log_probs=log_probs.sum(1,keepdim=True)
        
        return action,log_probs

In [ ]:
class Q(nn.Module):
    def __init__(self,state_dim,action_dim,hidden_dim):
        super(Q,self).__init__()
        self.q=nn.Sequential(
            nn.Linear(state_dim+action_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,1)
        )
        
    def forward(self,state,action):
        x=torch.cat([state,action],dim=-1)
        return self.q(x)

In [26]:
class Critic(nn.Module):
    def __init__(self,state_dim,hidden_dim):
        super(Critic,self).__init__()
        self.critic=nn.Sequential(
            nn.Linear(state_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,1)
        )
        
    def forward(self,state):
        return self.critic(state)

构建SAC

In [91]:
class SAC:
    def __init__(self,state_dim,action_dim,hidden_dim,
                 max_action,actor_lr=1e-3,value_lr=1e-3,
                 q_lr=1e-3,gamma=0.99,tau=0.005,memory_size=8196,batch_size=64):
        #读取参数
        self.state_dim=state_dim
        self.action_dim=action_dim
        self.hidden_dim=hidden_dim
        self.max_action=max_action
        self.actor_lr=actor_lr
        self.q_lr=q_lr
        self.value_lr=value_lr
        self.gamma=gamma
        self.tau=tau
        self.memory_size=memory_size
        self.batch_size=batch_size
        
        #初始化各部件
        ##回放缓存
        self.replay_buffer=ReplayBuffer(self.memory_size,self.state_dim,self.action_dim)
        ##策略网络
        self.actor=Actor(self.state_dim,self.action_dim,self.hidden_dim,self.max_action)
        #self.target_actor=Actor(self.state_dim,self.action_dim,self.hidden_dim,self.max_action)
        self.actor_optimizer=optim.Adam(self.actor.parameters(),lr=actor_lr)
        ##价值网络
        self.value=Critic(self.state_dim,self.hidden_dim)
        self.target_value=Critic(self.state_dim,self.hidden_dim)
        self.value_optimizer=optim.Adam(self.value.parameters(),lr=value_lr)
        ##动作价值网络
        self.qvalue=Q(self.state_dim,self.action_dim,self.hidden_dim)
        #self.target_qvalue=Q(self.state_dim,self.action_dim,self.hidden_dim)
        self.qvalue_optimizer=optim.Adam(self.qvalue.parameters(),lr=q_lr)
        
    def store(self,state,action,reward,next_state,done):
        self.replay_buffer.add(state,action,reward,next_state,done)
    
    def sample(self):
        state,action,reward,next_state,done=self.replay_buffer.sample(self.batch_size)
        state=torch.FloatTensor(state)
        action=torch.FloatTensor(action)
        reward=torch.FloatTensor(reward)
        next_state=torch.FloatTensor(next_state)
        done=torch.BoolTensor(done)
        return state,action,reward,next_state,done
    
    def select_action(self,state):
        state=torch.FloatTensor(state).unsqueeze(0)
        action,_=self.actor.select_action(state)
        return action.detach().numpy()[0]
    
    def update(self):
        #采样
        state,action,reward,next_state,done=self.sample()
        
        #计算alue值
        value=self.value(state).view(-1)
        next_value=self.target_value(next_state).view(-1)
        
        #选择动作
        action_sampled,log_prob=self.actor.select_action(state)
        log_prob=log_prob.view(-1)
        
        #计算q值
        q_value=self.qvalue(state,action_sampled).view(-1)
        
        #更新value网络
        self.value_optimizer.zero_grad()
        target_value=q_value-log_prob
        value_loss=0.5*F.mse_loss(value,target_value)
        value_loss.backward(retain_graph=True)
        self.value_optimizer.step()
        
        #更新Actor网络
        self.actor_optimizer.zero_grad()
        actor_loss=log_prob-q_value
        actor_loss=torch.mean(actor_loss)
        actor_loss.backward(retain_graph=True)
        self.actor_optimizer.step()
        
        #更新Q网络
        self.qvalue_optimizer.zero_grad()
        q_hat=reward+self.gamma*next_value
        q_value=self.qvalue(state,action)
        q_loss=F.mse_loss(q_value.view(-1),q_hat)
        q_loss.backward()
        self.qvalue_optimizer.step()
        
        self.update_parameters()
        
    def update_parameters(self):
        value_state_dict=dict(self.value.named_parameters())
        target_value_state_dict=dict(self.target_value.named_parameters())
        
        for name in value_state_dict:
            value_state_dict[name]=self.tau*value_state_dict[name].clone()+\
                (1-self.tau)*target_value_state_dict[name].clone()
        self.target_value.load_state_dict(value_state_dict)

测试init

In [ ]:
#初始化
agent=SAC(5,1,32,2)
#网络是否可以运行
state=torch.randn(5)
action,_=agent.actor(state)
value=agent.value(state)
qvalue=agent.qvalue(state,action)
#优化器是否可运行
##生成目标值
state=torch.randn(5)
action_,_=agent.actor(state)
value_=agent.value(state)
qvalue_=agent.qvalue(state,action_)
##计算loss
action_loss=F.mse_loss(action,action_)
value_loss=F.mse_loss(value,value_)
qvalue_loss=F.mse_loss(qvalue,qvalue_)
##清空梯度->反向传播->优化
action_loss.backward()
agent.actor_optimizer.step()
agent.actor_optimizer.zero_grad()
value_loss.backward(retain_graph=True)
agent.value_optimizer.step()
agent.value_optimizer.zero_grad()
qvalue_loss.backward()
agent.qvalue_optimizer.step()
agent.qvalue_optimizer.zero_grad()

测试rb模块

In [ ]:
agent=SAC(5,1,32,2)
for _ in range(10000):
    state=np.random.randn(5)
    action=np.random.randn(1)
    reward=np.random.randn(1)
    next_state=np.random.randn(5)
    done=np.random.randint(0,2)
    agent.store(state,action,reward,next_state,done)

这里输入需要维度

In [ ]:
ac=Actor(5,32,1)
action,log_probs=ac.select_action(torch.randn(5,5))
log_probs

在gym环境中测试

In [70]:
env=gym.make('Pendulum-v0')
state_dim=env.observation_space.shape[0]
action_dim=env.action_space.shape[0]

需要用unsqueeze(0)将tensor升维

In [92]:
agent=SAC(state_dim,action_dim,hidden_dim=32,max_action=env.action_space.high.item())
a=agent.select_action(torch.FloatTensor(env.reset()).unsqueeze(0))

Actor网络中logProb似乎有些问题

In [103]:

for it in range(100):
    done=False
    state=env.reset()
    while not done:
        action=agent.select_action(state)
        state_,r,done,_=env.step(action)
        agent.store(state,action,r,state_,done)
        agent.update()
        state=state_
    if it%10==0:
        print('iteration：',it)

iteration： 0
iteration： 10
iteration： 20
iteration： 30
iteration： 40
iteration： 50
iteration： 60
iteration： 70
iteration： 80
iteration： 90


Gym环境跑通！

下一步开始适配Aurora环境

In [99]:
import network_sim

In [108]:
env=gym.make('PccNs-v0')
state_dim=env.observation_space.shape[0]
action_dim=env.action_space.shape[0]

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


/home/data/envs/IL/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [119]:
AuroraAgent=SAC(state_dim,action_dim,32,env.action_space.high[0])

In [ ]:
epsilon=0.1
for i in range(100):
    done=False
    s=env.reset()
    while not done:
        action=AuroraAgent.select_action(s)
        s_,r,done,_=env.step(action)
        AuroraAgent.store(s,action,r,s_,done)
        AuroraAgent.update()
        s=s_
    if i%10==0:
        print('iteration:',i)

In [122]:
env.observation_space.sample()

array([ 8.0688076e+00,  9.8935059e+03,  2.9261279e+02,  9.9961920e+00,
        5.3750400e+03,  3.5672318e+02, -6.4626765e-01,  8.5764717e+03,
        3.8121045e+02,  1.3743095e+00,  2.1900139e+03,  1.8200874e+02,
        6.1614542e+00,  8.8220762e+03,  8.8058185e+02, -7.2015804e-01,
        2.1096001e+03,  9.1302472e+02, -7.2793168e-01,  6.4749634e+03,
        2.8407443e+02,  5.0648036e+00,  1.6514847e+03,  3.1797876e+02,
        2.9469061e+00,  6.3996619e+02,  8.0839709e+02,  7.6744790e+00,
        6.8565776e+03,  4.6947217e+02], dtype=float32)